# Preconditioner Series: Introduction to Preconditioners

When solving linear systems such as $Ax=b$ iteratively, the condition number, $\kappa$, of $A$ is indicative of the rate of convergence. Preconditioners ($P$), then, "massage" these linear systems to have lower conditioner numbers, and, thus, higher rates of convergence.

$$ \kappa(P^{-1}A)\ \text{,}\  \kappa(AP^{-1}) << \kappa(A) $$

The linear system can be right or left preconditioned:

Right preconditioned system
1. Introduce $P^{-1}P=I$
$$ AP^{-1}Px=b $$
2. Solve for $y$
$$ AP^{-1}y=b $$
3. Solve for $x$
$$ Px=y $$

Left preconditioned system
1. Multiply both sides by $P^{-1}$
$$ P^{-1}Ax=P^{-1}b $$
2. Gather terms on left side
$$ P^{-1}(Ax-b)=0 $$

One helpful way of explaining preconditioners in not-so-rigorous terminology was that **$P^{-1}$ "looks like" $A^{-1}$**. Of course, if $P^{-1}=A^{-1}$, then the solution to the linear system would be trivially easily to calculate.

## Imports

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

Here, I demonstrate the effectiveness of preconditioning by solving a linear system before and after preconditioning. Though a variety of iterative methods exist, I will employ one of the simplest - the Jacobi method.

## Jacobi preconditioner

The Jacobi preconditioner is quite simple:

<img src="img/jacobi_matrixform.png" alt="term-document matrix" style="width: 100%"/>

The last equation can be expressed to emphasize the iterative nature of this method:

$$\begin{align} x^{(k)} &= D^{-1}(L+U)x^{(k-1)}+D^{-1}b \\
x^{(k)} &= D^{-1}(b-Rx^{(k-1)})\end{align}$$

Here, $R=-(L+U)$

Typically, calculating the inverse of a matrix is unfavorable; however, because $D$ is a diagonal matrix, its inverse is trivially easy to calculate: it is simply the reciprocal of the diagonal elements.

### Jacobi preconditioner implementation

In [126]:
def converged(x, x_new, rtol, atol):
    if np.linalg.norm(x - x_new) / np.linalg.norm(x_new) < rtol or np.linalg.norm(x - x_new) < atol:
        return True
    else: return False

In [127]:
def jacobi_method(A, x0, b, rtol=1e-5, atol=1e-8, max_iter=10000):
    
    # find inverse of diagonal matrix and off-diagonal matrix
    D_inv, R = np.diag(np.reciprocal(np.diagonal(A).astype(float))), A - np.diag(np.diagonal(A))
    
    i = 0
    x = x0
    
    # loop for max_iter iterations or until converged
    while (i < max_iter):
        x_new = D_inv @ (b - R @ x)
        
        # test convergence
#         if (np.allclose(x, x_new, rtol, atol)):
        if (converged(x, x_new, rtol, atol)):
            return x_new
        
        # prep for next iteration
        x = x_new
        i += 1
        
    # solution did not converge
    print('Maximum number of iterations reached. Did not converge.')

First we create a matrix $A$.

In [128]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])

Now, we create a vector $b$.

In [129]:
b = np.array([6., 25., -11., 15.])

Now, we iteratively solve for $x$.

The first step is to provide an initial estimate for $x$, which I'll call $x^{(0)}$.

In [130]:
x0 = np.zeros_like(b); x0

array([0., 0., 0., 0.])

Now, we can pass $A$, $x^{(0)}$, and $b$ and begin iteratively solving for $x$.

In [133]:
%timeit x = jacobi_method(A, x0, b, rtol=1e-4, atol=1e-4, max_iter=1000); x

1000 loops, best of 3: 269 µs per loop


## Resources

[Iterative Methods for Sparse Linear System](https://www.amazon.com/Iterative-Methods-Sparse-Linear-Systems/dp/0898715342)

[The Jacobi and Gauss-Seidel Iterative Methods](https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf)